### Importing Libraties

In [7]:
import pandas

## Policy Network Implementation

In [3]:
# x - vector, that holds the preprocessed pixel information
# W1 and W2 - matrices, that we initialize randomly

h = np.dot(W1, x) # compute hidden layer neuron activations
h[h<0] = 0 # ReLU nonlinearity: threshold at zero
logp = np.dot(W2, h) # compute log probability  of going up
p = 1.0 / (1.0 + np.exp(-logp)) # sigmoid function (gives probability of going up)


NameError: name 'W1' is not defined

### Preprocessing
#### We’ll actually feed subtraction of current and last frame so that it can detect motion

## Credit Assignment Problem

### Policy Gradients

#### We’ll use a 2-layer neural network that takes the raw image pixels (100,800 numbers total (210 * 160 * 3)), and produces a single number indicating the probability of going UP

#### The neurons in the hidden layer (which have their weights arranged along the rows of W1) can detect various game scenarious (e.g. the ball is in the top, and our paddle is in the middle), and the weights in W2 can then decide if in each case we should be going UP or DOWN

#### Our policy network network calculated probability of going UP as 30% (logprob -1.2) and DOWN as 70% (-0.36).
#### We will sample now sample an action from this distribution; E.g. suppose we sample DOWN, and we will execute it in the game. At this point notice one interesting fact: We could immediately fill in a gradient of 1.0 for DOWN as we did in supervised learning, and find the gradient vector that would encourage the network to be slightly more likely to do the DOWN action in the future.
#### So we can immediately evaluate this gradient and that's great, but the problem is that at least for now we do not yet know if going DOWN is good. But the critical point is that that's okay, because we can simply wait a bit and see!
#### For example in Pong we could wait until the end of the game, then take the reward we get and enter that scalar as the gradient for the action we have taken (DOWN in this case).
#### In the example below, going DOWN ended up to us losing the game (-1 reward). So if we fill in -1 for log probability of DOWN and do backprop we will find a gradient that discourages the network to take the DOWN action for that input in the future.

## Training Protocol

#### We will initialize the policy network with some W1, W2 and play 100 games of Pong (we call these policy “rollouts”).
#### Lets assume that each game is made up of 200 frames so in total we’ve made 20,000 decisions for going UP or DOWN, and for each one of these we know the parameter gradient, which tells us how we should change the parameters if we wanted to encourage that decision in that state in the future.
#### All that remains now is to label every decision we've made as good or bad.
#### For example suppose we won 12 games and lost 88. We'll take all 200 * 12 = 2400 decisions we made in the winning games and do a positive update (filling in a +1.0 in the gradient for the sampled action, doing backprop, and parameter update encouraging the actions we picked in all those states.
#### Now we play another 100 games with our new, slightly improved policy and rinse and repeat.

In [9]:
import sys
print(sys.version)

3.8.2 (tags/v3.8.2:7b3ab59, Feb 25 2020, 23:03:10) [MSC v.1916 64 bit (AMD64)]


[0.00113139 0.00077257 0.01165723 0.00539009]
[ 0.00114684 -0.19451461  0.01176504  0.3017281 ]
[-0.00274345  0.0004377   0.0177996   0.01277872]
[-0.00273469 -0.19493494  0.01805517  0.31102405]
[-0.00663339 -0.3903094   0.02427565  0.60934592]
[-0.01443958 -0.58576215  0.03646257  0.909575  ]
[-0.02615482 -0.39115216  0.05465407  0.62857167]
[-0.03397787 -0.58699255  0.06722551  0.93795392]
[-0.04571772 -0.78295335  0.08598458  1.25098059]
[-0.06137678 -0.97906543  0.1110042   1.56931027]
[-0.08095809 -0.78543011  0.1423904   1.31321316]
[-0.0966667  -0.98203819  0.16865466  1.64686212]
[-0.11630746 -1.17868372  0.20159191  1.986995  ]
Episode finished after 13 timesteps
[ 0.03715637  0.03046338 -0.00673433  0.04491878]
[ 0.03776564  0.22568125 -0.00583596 -0.24988126]
[ 0.04227926  0.42088605 -0.01083358 -0.54439924]
[ 0.05069698  0.225918   -0.02172157 -0.25514933]
[ 0.05521534  0.42134325 -0.02682455 -0.55460369]
[ 0.06364221  0.61683138 -0.03791663 -0.85561586]
[ 0.07597883  0.81